<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/hybrid_parity_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane torch tqdm

In [ ]:
#!/usr/bin/env python3
"""
hybrid_parity.py

A self-contained PyTorch + PennyLane script to train a 4-bit parity classifier
using a hybrid quantum-classical model with:
  - Angle data embedding
  - BasicEntanglerLayers variational circuit (8 layers, full entanglement)
  - Classical residual block (no in-place ops)
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pennylane as qml
from pennylane import numpy as np


# 1. Configuration
n_qubits = 4
n_layers = 8

# Built-in device
dev = qml.device("default.qubit", wires=n_qubits)


# 2. QNode (must name first arg "inputs" for TorchLayer)
@qml.qnode(dev, interface="torch", diff_method="backprop")
def parity_qnode(inputs, weights):
    # AngleEmbedding only
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation="X")

    # Full-entangling variational circuit
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))

    # Measure Z on each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


# Wrap into a TorchLayer
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.TorchLayer(parity_qnode, weight_shapes)


# 3. Hybrid model with a classical residual block (no in-place add)
class HybridParityModel(nn.Module):
    def __init__(self, hidden_dim: int = 32):
        super().__init__()
        self.qlayer     = qlayer
        self.res_fc1    = nn.Linear(n_qubits, hidden_dim)
        self.res_fc2    = nn.Linear(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, 2)
        self.relu       = nn.ReLU()

    def forward(self, x):
        # Quantum feature map
        q_out = self.qlayer(x)

        # Classical residual block
        h      = self.relu(self.res_fc1(q_out))
        h_res  = self.relu(self.res_fc2(h))
        h      = h + h_res       # <- out-of-place addition

        # Return two-class logits
        return self.classifier(h)


# 4. Parity dataset generator
def generate_parity_data(n_samples: int = 1024):
    X = np.random.randint(0, 2, size=(n_samples, n_qubits))
    X = 2 * X - 1                  # map {0,1} → {-1,+1}
    y = (np.sum(X == 1, axis=1) % 2).astype(int)
    return (
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long),
    )


def train():
    # Instantiate model, optimizer, criterion
    model     = HybridParityModel(hidden_dim=32)
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    # Prepare data
    X_train, y_train = generate_parity_data(2000)
    X_test,  y_test  = generate_parity_data(500)

    train_loader = DataLoader(TensorDataset(X_train, y_train),
                              batch_size=32, shuffle=True)
    test_loader  = DataLoader(TensorDataset(X_test, y_test),
                              batch_size=64)

    # Training loop
    for epoch in range(1, 51):
        model.train()
        total_loss, correct = 0.0, 0

        for Xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(Xb)
            loss   = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * Xb.size(0)
            correct    += (logits.argmax(dim=1) == yb).sum().item()

        train_acc  = correct / len(train_loader.dataset)
        train_loss = total_loss / len(train_loader.dataset)

        # Evaluation
        model.eval()
        test_correct = 0
        with torch.no_grad():
            for Xb, yb in test_loader:
                preds        = model(Xb).argmax(dim=1)
                test_correct += (preds == yb).sum().item()
        test_acc = test_correct / len(test_loader.dataset)

        print(
            f"Epoch {epoch:2d} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Train Acc: {train_acc:.3f} | "
            f"Test Acc: {test_acc:.3f}"
        )


if __name__ == "__main__":
    train()